In [47]:
import math
import numpy as np
import pandas as pd
from numpy.random import seed
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow import set_random_seed
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import Adam
from JanetLSTM import JANET
from keras.callbacks import ModelCheckpoint

In [62]:
file_path = './data/jobs.csv'
test_size = 0.2
cv_size = 0.2

N = 9
batch_size = 100
num_classes = 10
epochs = 100
hidden_units = 128 
dropout_prob=0.5
learning_rate = 0.001

# Set seeds to ensure same output results
# seed(101)
# set_random_seed(model_seed)

df = pd.read_csv('./data/jobs.csv', header=0, index_col=0)
df.sort_values(by='submit_time', inplace=True, ascending=True)
print(df)

               duration  cpu    memory  job_id  task_id  instances_num  disk
submit_time                                                                 
0            131.964467  0.5  0.008935    1750     9550            197     0
1             58.142857  0.5  0.007895    1750     9551             77     0
2            109.452675  0.5  0.008997    1750     9552            243     0
3             38.068966  0.5  0.016007    1750     9553             29     0
4             30.615385  0.5  0.013356    1750     9554             13     0
...                 ...  ...       ...     ...      ...            ...   ...
31751          5.000000  0.5  0.004121    7025    41415              1     0
31752         85.000000  0.5  0.016338    3318    20691              1     0
31753          7.000000  0.5  0.008055    3318    20692              2     0
31754         39.000000  0.5  0.005736    3318    20693              3     0
31755          1.583333  0.5  0.004173   12700    79294           6818     0

In [63]:
#函数1：计算MAPE指标
def get_mape(y_true, y_pred):
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [64]:
def get_x_y(data, N, offset):
    """
    Split data into x (features) and y (target)
    """
    x, y = [], []
    for i in range(offset, len(data)):
        x.append(data[i - N:i])
        y.append(data[i,0])
    x = np.array(x)
    y = np.array(y)

    return x, y

In [67]:
def train_pred_eval_model(x_train_scaled, \
                          y_train_scaled, \
                          x_cv_scaled, \
                          y_cv_scaled, \
                          scaler, \
                          lstm_units=50, \
                          dropout_prob=0.5, \
                          optimizer='adam', \
                          epochs=1, \
                          batch_size=1):
    model = Sequential()
    model.add(JANET(units=lstm_units,  input_shape=x_train_scaled.shape[1:]))
    model.add(Dropout(dropout_prob))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    rmsprop = Adam(lr=learning_rate, amsgrad=True)
    
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])
    history = model.fit(x_train_scaled, y_train_scaled, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(x_cv_scaled, y_cv_scaled), 
                        verbose=0, 
                        shuffle=False,
                        callbacks=[ModelCheckpoint('weights/model.h5', monitor='val_acc',
                                                   save_best_only=True, save_weights_only=True, mode='max')])
    # 进行预测：
    pred_y_cv_scaled = model.predict(x_cv_scaled)
    print(pred_y_cv_scaled)

    #借助x，还原pred_y_cv
    X_cv_reduce = x_cv_scaled[:, 0:1, :]  # 去掉time_lag
    print(X_cv_reduce)
    print(X_cv_reduce.shape)
    X_cv_reduce = np.reshape(X_cv_reduce, [len(X_cv_reduce), -1])
    print(X_cv_reduce)
    print(X_cv_reduce.shape)
    pred_y_cv = np.concatenate((pred_y_cv_scaled, X_cv_reduce[:, 1:]), axis=1)  # 主要目的： （y+X）匹配Scaler做逆变换
    print(pred_y_cv.shape)
    pred_y_cv = scaler.inverse_transform(pred_y_cv)
    pred_y_cv=pred_y_cv[:,0]
    
    #借助x，还原y_cv
    # invert scaling for actual 原始矩阵
    y_cv_scaled = y_cv_scaled.reshape((len(y_cv_scaled), 1))
    y_cv = np.concatenate((y_cv_scaled, X_cv_reduce[:, 1:]), axis=1)  # 还原原始test_y

    y_cv = scaler.inverse_transform(y_cv)
    y_cv = y_cv[:, 0]

    # 计算 RMSE and MAPE
    rmse = math.sqrt(mean_squared_error(y_cv, pred_y_cv))
    mape = get_mape(y_cv, pred_y_cv)

    return rmse, mape, pred_y_cv,history

In [15]:
# Get sizes of each of the datasets
num_cv = int(cv_size*len(df))
num_test = int(test_size*len(df))
num_train = len(df) - num_cv - num_test
print("num_train = " + str(num_train))
print("num_cv = " + str(num_cv))
print("num_test = " + str(num_test))

num_train = 19054
num_cv = 6351
num_test = 6351


In [49]:
train = df[:num_train]
cv = df[num_train:num_train + num_cv]
train_cv = df[:num_train + num_cv]
test = df[num_train + num_cv:]

N_opt=5   #窗口大小
batch_size = 100
num_classes = 10
epochs = 100
hidden_units = 128 
dropout_prob=0.5

scaler_final = MinMaxScaler(feature_range=(0, 1))
train_cv_scaled_final = scaler_final.fit_transform(np.array(train_cv[['duration', 'cpu','memory','instances_num']]).reshape(-1,4))
# 训练集划分：Split train_cv into x（符合LSTM输入形式） and y
x_train_cv_scaled, y_train_cv_scaled = get_x_y(train_cv_scaled_final, N_opt, N_opt)
#测试集划分，因为test的窗口涉及cv部分的数据，所以从df中截取比较好
# 对整个数据集进行缩放
df_scaled  = scaler_final.transform(np.array(df[['duration', 'cpu','memory','instances_num']]).reshape(-1,4))
x_test_scaled, y_test_scaled = get_x_y(df_scaled, N_opt, num_train+num_cv)

In [68]:
rmse, mape, pred_test_y,history = train_pred_eval_model(x_train_cv_scaled, \
                                        y_train_cv_scaled, \
                                        x_test_scaled, \
                                        y_test_scaled, \
                                        scaler_final, \
                                        lstm_units=lstm_units_opt, \
                                        dropout_prob=dropout_prob_opt, \
                                        optimizer=optimizer_opt, \
                                        epochs=epochs_opt, \
                                        batch_size=batch_size)
# 计算 RMSE
print("RMSE on test set = %0.3f" % rmse)

# 计算 MAPE
print("MAPE on test set = %0.3f%%" % mape)


pyplot.plot(history.history['loss'], label='train loss')
pyplot.plot(history.history['val_loss'], label='test loss')
pyplot.legend()
pyplot.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
janet_16 (JANET)             (None, 50)                5500      
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                510       
Total params: 6,010
Trainable params: 6,010
Non-trainable params: 0
_________________________________________________________________
[[9.9998760e-01 9.0762278e-06 3.5482887e-07 ... 1.4490139e-07
  1.0399405e-06 2.4849058e-07]
 [9.9998343e-01 1.1811681e-05 4.8589430e-07 ... 2.0072446e-07
  1.4090222e-06 3.4289886e-07]
 [9.9998474e-01 1.0874536e-05 4.3977192e-07 ... 1.8027990e-07
  1.2775894e-06 3.0841210e-07]
 ...
 [9.9998820e-01 8.6074679e-06 3.3584294e-07 ... 1.3658658e-07
  9.7914187e-07 2.3268618e-07]
 [9.9998558e-01 1.0332305e-

ValueError: operands could not be broadcast together with shapes (6351,13) (4,) (6351,13) 